In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input	

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator

Using CNTK backend


In [2]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')

In [3]:
target_train=train['is_iceberg']
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_band_3=(X_band_1+X_band_2)/2
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)



X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)

batch_size=64
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=2):
   es = EarlyStopping('val_loss', patience=10, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]


def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    

    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model


#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        model_extractfeatures = Model(input=galaxyModel.input, output=galaxyModel.get_layer('fc2').output)
        print "here"
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-3-a6a74ef3afc4>, line 130)

In [4]:
preds=myAngleCV(X_train, X_angle, X_test)


===================FOLD= 0


C:\Users\PSaraswat\AppData\Local\Continuum\anaconda3\envs\dlwin36\lib\site-packages\ipykernel_launcher.py:75: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Composite(..., inputs=[Input('in...)`


Epoch 1/100


C:\Users\PSaraswat\AppData\Local\Continuum\anaconda3\envs\dlwin36\lib\site-packages\cntk\core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input4") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


24/24 [==============================] - 272s 11s/step - loss: 0.7234 - acc: 0.6035 - val_loss: 0.3642 - val_acc: 0.7963
Epoch 2/100
24/24 [==============================] - 233s 10s/step - loss: 0.4244 - acc: 0.7984 - val_loss: 0.3271 - val_acc: 0.8374
Epoch 3/100
24/24 [==============================] - 231s 10s/step - loss: 0.3587 - acc: 0.8370 - val_loss: 0.2467 - val_acc: 0.8860
Epoch 4/100
24/24 [==============================] - 231s 10s/step - loss: 0.2894 - acc: 0.8724 - val_loss: 0.2251 - val_acc: 0.8991
Epoch 5/100
24/24 [==============================] - 234s 10s/step - loss: 0.2867 - acc: 0.8790 - val_loss: 0.2250 - val_acc: 0.8972
Epoch 6/100
24/24 [==============================] - 245s 10s/step - loss: 0.2399 - acc: 0.8898 - val_loss: 0.2303 - val_acc: 0.8935
Epoch 7/100
24/24 [==============================] - 231s 10s/step - loss: 0.2511 - acc: 0.8937 - val_loss: 0.2269 - val_acc: 0.9065
Epoch 8/100
24/24 [==============================] - 230s 10s/step - loss: 0.2387

C:\Users\PSaraswat\AppData\Local\Continuum\anaconda3\envs\dlwin36\lib\site-packages\cntk\core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input2622") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


24/24 [==============================] - 264s 11s/step - loss: 0.8519 - acc: 0.5741 - val_loss: 0.5259 - val_acc: 0.7028
Epoch 2/100
24/24 [==============================] - 288s 12s/step - loss: 0.5135 - acc: 0.7351 - val_loss: 0.3174 - val_acc: 0.8654
Epoch 3/100
24/24 [==============================] - 247s 10s/step - loss: 0.3639 - acc: 0.8245 - val_loss: 0.2452 - val_acc: 0.8972
Epoch 4/100
24/24 [==============================] - 250s 10s/step - loss: 0.3218 - acc: 0.8539 - val_loss: 0.2854 - val_acc: 0.8785
Epoch 5/100
24/24 [==============================] - 247s 10s/step - loss: 0.2987 - acc: 0.8724 - val_loss: 0.2263 - val_acc: 0.9084
Epoch 6/100
24/24 [==============================] - 247s 10s/step - loss: 0.2686 - acc: 0.8816 - val_loss: 0.2086 - val_acc: 0.9140
Epoch 7/100
24/24 [==============================] - 246s 10s/step - loss: 0.2556 - acc: 0.8920 - val_loss: 0.2362 - val_acc: 0.8785
Epoch 8/100
24/24 [==============================] - 245s 10s/step - loss: 0.2367

C:\Users\PSaraswat\AppData\Local\Continuum\anaconda3\envs\dlwin36\lib\site-packages\cntk\core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input5240") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


23/24 [===========================>..] - ETA: 9s - loss: 0.6837 - acc: 0.6232 

RuntimeError: mkl err (-3)


[CALL STACK]
    > 000007FEB28A71A8 (SymFromAddr() error: The specified module could not be found.)
    - 000007FEB28A9F60 (SymFromAddr() error: The specified module could not be found.)
    - 000007FEB28A8E03 (SymFromAddr() error: The specified module could not be found.)
    - 000007FEB28A7AE1 (SymFromAddr() error: The specified module could not be found.)
    - 000007FEB8079809 (SymFromAddr() error: The specified module could not be found.)
    - 000007FEB802703D (SymFromAddr() error: The specified module could not be found.)
    - 000007FEB8144126 (SymFromAddr() error: The specified module could not be found.)
    - 000007FEB8032B7F (SymFromAddr() error: The specified module could not be found.)
    - 000007FEB814C08E (SymFromAddr() error: The specified module could not be found.)
    - 000007FEB818BE64 (SymFromAddr() error: The specified module could not be found.)
    - 000007FEC0A0CE47 (SymFromAddr() error: The specified module could not be found.)
 

In [ ]:
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('sub2.csv', index=False)